In [53]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [54]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
from bs4 import BeautifulSoup

In [56]:
import warnings
warnings.filterwarnings('ignore')

Use BeautifulSoup to read XML files

In [57]:
positive_reviews = BeautifulSoup(open('/content/positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')

In [58]:
import string

def tokenize(text):
  tokens = text.lower()
  tokens = nltk.tokenize.word_tokenize(tokens)
  return tokens

In [59]:
trigram = {}

In [60]:
positive_tokenized_reviews = []
for review in positive_reviews:
  tokens = tokenize(review.text)
  positive_tokenized_reviews.append(tokens)

  for i in range(1, len(tokens)-1):
    key = (tokens[i-1],tokens[i+1])
    if key not in trigram:
      trigram[key] = {}
      trigram[key][tokens[i]] = 1
    elif tokens[i] not in trigram[key]:
      trigram[key][tokens[i]] = 1
    else:
      trigram[key][tokens[i]] += 1

In [61]:
import itertools

K = 8
res = dict(itertools.islice(trigram.items(), K))
res

{('sphere', 'michael'): {'by': 1},
 ('by', 'crichton'): {'michael': 1},
 ('michael', 'is'): {'crichton': 1, 'sheen': 1, 'hauge': 1},
 ('crichton', 'an'): {'is': 1, 'takes': 1},
 ('is', 'excellant'): {'an': 1},
 ('an', 'novel'): {'excellant': 1, 'engaging': 1, 'award-winning': 1},
 ('excellant', '.'): {'novel': 1},
 ('novel', 'this'): {'.': 2, 'before': 1, 'of': 1}}

In [62]:
for key in trigram:
  total = sum(trigram[key].values())
  for word in trigram[key]:
    trigram[key][word] /= total

In [63]:
import itertools

K = 8
res = dict(itertools.islice(trigram.items(), K))
res

{('sphere', 'michael'): {'by': 1.0},
 ('by', 'crichton'): {'michael': 1.0},
 ('michael', 'is'): {'crichton': 0.3333333333333333,
  'sheen': 0.3333333333333333,
  'hauge': 0.3333333333333333},
 ('crichton', 'an'): {'is': 0.5, 'takes': 0.5},
 ('is', 'excellant'): {'an': 1.0},
 ('an', 'novel'): {'excellant': 0.3333333333333333,
  'engaging': 0.3333333333333333,
  'award-winning': 0.3333333333333333},
 ('excellant', '.'): {'novel': 1.0},
 ('novel', 'this'): {'.': 0.5, 'before': 0.25, 'of': 0.25}}

In [64]:
def sample_word(dictionary):
  p0 = np.random.random()
  cumulative = 0
  for word, prob in dictionary.items():
    cumulative += prob
    if p0 < cumulative:
      return word

In [67]:
import random

def spin_article(article):
    review = random.choice(article)
    original = review.text
    print("Original review: \n", original, "\n-----------------\n")
    tokens = tokenize(original)
    for i in range(1, len(tokens)-1):
      if random.random() < 0.2:
        key = (tokens[i-1], tokens[i+1])
        if key in trigram:
          new_word = sample_word(trigram[key])
          tokens[i] = new_word

    new_review = " ".join(tokens).replace(" .", ".").replace("$ ", "$").replace(" ,", ",").replace(" !", "!").replace(" ?", "?").replace(" '", "'")
    print("New review: \n", new_review)

In [68]:
spin_article(positive_reviews)

Original review: 
 
I am admittedly a terrible cook, so I follow recipes to the letter, often with nonmagical results. With Julia Child in my kitchen, however, things are looking up! Her recipe for roast chicken was so exact that even I could not mess it up. That and the sauteed beef steaks are now my go-to dinners because they are easy and reliable.  I've also had great success with the quiche and biscuit recipes and look forward to trying breadmaking for the first time.  This book is essential for anyone out on his or her own for the first time, newlyweds, or people like me who just don't get it (but really want to)
 
-----------------

New review: 
 i am admittedly a terrible cook, so i follow recipes to the letter, often with nonmagical results. with julia child in my favorite, however, things are looking up! her recipe for roast chicken was so exact that even i could not mess it up. that and the sauteed beef steaks are now my go-to dinners because they are overlong and calm. i've 